## Q&A on a Website content through LLM
#### Retrieval augmented generation, or RAG
- Load web content using `WebBaseLoader` and `bs4`
- Create docs and divide into chunks using `RecursiveCharacterTextSplitter`
- Initialize embedding `OpenAIEmbeddings` to create vector embeddings
- Store to document chunks as vector store db (`Chroma`, `FAISS`) 
- Import and intialize LLM - `ChatOpenAI` with `gpt-4o` model
- Create prompt template
- Create chain to

**PromptTemplate** is used to dynamically generate LLM prompts by embedding variables into a predefined structure
- ChatPromptTemplate.`from_template(prompt)`
    - Creates a chat template consisting of a single message assumed to be from the human.
    - `prompt = """Suggest a restaurant name in {country} that serves {cuisine} food."""`
- ChatPromptTemplate.`from_messages(prompt)`
    - Create a chat prompt template from a variety of message formats.
    - `prompt= [("system", "Act like a doctor and answer my questions"),("user", "{input}")`

**Retriever** 
- An interface for fetching or retrieving relevant data from documents(chunks) or vector store
- VectorStore objects are not Runnables so cannot be integrated directly with LCEL, Retriver are Runnables which acts as bridge

**Chain** 
- Refers to a sequence of steps or components that are linked together to perform a more complex task using LLMs.
- Series of `Runnable` that are connected to ble executed together
- Langchain provides LCEL - Langchain Expression Language to create chains easily with Runnables
    - Example - `prompt-template | llm | output-parser`
- https://python.langchain.com/v0.1/docs/modules/chains/


#### Load -> Chunk -> Embed -> Store

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from IPython.display import Markdown

loader = WebBaseLoader("https://www.sobha.com/blog/bangalore-real-estate-market-trends/")
documents = loader.load()

sp = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_chunks = sp.split_documents(documents)

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(doc_chunks, embeddings)

Markdown(db.similarity_search('Why is Bangalore rent so costly?')[0].page_content)


7. Why is real estate booming in Bangalore? 

Real estate is booming in Bangalore due to demand driven by the city's expanding population and relatively lower expense of living. According to Bangalore real estate trends 2024, many people are looking for opulent and easily accessible dwelling options. 





8. Will flat prices go up in Bangalore 

Yes, flat prices could go up in Bangalore, as the city has witnessed the most increase in house costs, at 7%, while Mumbai saw a 6% increase. In contrast, prices in Chennai and Hyderabad increased by 5% each in the first quarter of 2023. 





9. Why is house rent so expensive in Bangalore? 

House rent is so expensive in Bangalore due to employees’ return to work-from-office, creating a spike in demand for housing. Another reason is higher commute times that have forced professionals to pick residences close to their workplaces, typically concentrated in particular locations, increasing the demand for apartments.

In [12]:
Markdown(db.similarity_search('who is the foreign minister of Bangladesh?')[0].page_content)

Comprehensive Urban Plan
The Revised Master Plan 2041 aims to transform Bangalore into a ‘compact city’ by promoting development around public transport systems. This plan includes significant upgrades to metro and road infrastructure, ensuring enhanced support for the city’s growth.
These infrastructure projects will drive the demand for housing, with estimates suggesting that more than 100,000 new residential units will be required by 2025. Improved connectivity and reduced travel time will make the city an even more attractive destination for businesses and residents alike.

#### Chat Prompt template

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the given context:
    <context>
    {context}
    </context>
    """
)

#### Document Chain
- *Processes a series of documents or text inputs to produce an output in stages*
- *Useful for summarization, translation, or other step-by-step transformations.*
- Stages - `Prompt Template -> LLM -> Output Parser`

In [14]:
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = ChatOpenAI(model="gpt-4o")
doc_chain = create_stuff_documents_chain(llm, prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the given context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021CF8A6AC20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021CF8A2F280>, root_client=<openai.OpenAI object at 0x0000021CF8A69D20>, root_async_client=<openai.AsyncOpenAI object at 0x0000021CF8A6AC50>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': '

#### Vector Store Retriver and Retrival Chain
- *To retrieve the context information through a Retriver (here it's from vector store db)*

In [15]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
# Used to retrieve from vector store
chain = create_retrieval_chain(retriever, doc_chain)
chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021CF8A2D180>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the given context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
      

#### Query

In [16]:
response = chain.invoke({"input": "Is Bangalore rent going increase?"})
Markdown(response['answer'])

Real estate is booming in Bangalore due to the city's expanding population and the relatively lower expense of living. Additionally, there is a high demand for opulent and easily accessible dwelling options, as noted in the Bangalore real estate trends for 2024.

In [17]:
response = chain.invoke({"input": "Which areas are good for residential property investments?"})
Markdown(response['answer'])

Based on the provided context, here are the answers to potential questions about Bangalore's real estate market in 2024:

1. **What are the anticipated price trends in Bangalore's property market for 2024?**
   - Anticipated price trends indicate a steady rise in both residential and commercial property segments, especially in well-connected micro-markets.

2. **Which areas in Bangalore are currently experiencing substantial growth and attracting considerable investment?**
   - Areas like Whitefield, Sarjapur Road, North Bangalore, Bellandur, and Yelahanka are experiencing substantial growth and attracting considerable investment due to improved connectivity and infrastructure.

3. **What is the significance of sustainability in Bangalore's new real estate projects?**
   - Eco-friendly features and sustainable development are becoming increasingly important, with many new projects incorporating green building practices and renewable energy sources.

4. **How is technology impacting Bangalore's real estate market?**
   - Technology is playing a pivotal role in the market with the rise of PropTech, which includes virtual tours, online transactions, and digital property management solutions becoming more prevalent.

5. **What trends are driving the demand for affordable housing in Bangalore?**
   - The demand for affordable housing continues to grow, supported by government initiatives and incentives aimed at making housing more accessible to the middle and lower-income segments.

6. **What are the key infrastructure developments in Bangalore that are expected to boost real estate values?**
   - Key infrastructure developments include new metro lines and suburban rail corridors. Notable projects are the Blue Line metro connecting Silk Board to Kempegowda International Airport (expected to be completed by mid-2026) and the Peripheral Ring Road encircling the city.

7. **Which locations are considered promising investment zones in Bangalore for 2024?**
   - In 2024, promising investment zones include Sarjapur Road, Banashankari, Devanahalli, Thanisandra, and Panathur, due to their proximity to business hubs and strong infrastructure.

8. **What is the expected rental yield in Bangalore's real estate market for 2024?**
   - The expected rental yields in Bangalore's real estate market average between 5% and 7%.

9. **How does Bangalore's IT industry influence its real estate market?**
   - Bangalore's robust IT industry drives economic growth, boosting housing demand and making the city an attractive destination for real estate investments due to excellent rental yields.

10. **What type of real estate developments are contributing to the boom in Bangalore's market in 2024?**
    - High-end office spaces, luxury homes, new roads, and metro lines are contributing to the boom in Bangalore's real estate market in 2024.

In [18]:
response = chain.invoke({"input": "Which are major infra developments coming up?"})
Markdown(response['answer'])

How will the ongoing infrastructure developments in Bangalore impact the real estate market and connectivity in the city?

The ongoing infrastructure developments in Bangalore are set to significantly impact the real estate market and connectivity within the city. Major projects such as new metro lines, suburban rail corridors, and expressways (like the Bengaluru-Chennai Expressway and Bangalore-Mysore Expressway) will enhance connectivity, reduce travel time, and alleviate congestion. These improvements will make office locations more accessible and attractive, thereby boosting commercial real estate values. Additionally, the development of satellite cities and the Peripheral Ring Road will support the city's expanding population and drive property demand, particularly in emerging hotspots like Sarjapur Road, Bellandur, and Yelahanka. Overall, these infrastructure upgrades will likely lead to increased investment and development in various neighborhoods, making Bangalore an even more appealing destination for both businesses and residents.

In [32]:
for chunk in chain.stream({"input": "Which are major infra developments coming up?"}):
    print(chunk, sep="", flush=True)

{'input': 'Which are major infra developments coming up?'}


{'context': [Document(metadata={'source': 'https://www.sobha.com/blog/bangalore-real-estate-market-trends/', 'title': 'Bangalore Real Estate Market Trends 2024', 'description': 'Discover the latest trends shaping the Bangalore real estate market in 2024. Stay informed for smart investment decisions.', 'language': 'en-US'}, page_content='Affordable Housing: \nThis demand continues to grow, supported by government initiatives and incentives aimed at making housing more accessible to the middle and lower-income segments.\n\nImpact of Infrastructure Development: \nMajor projects such as the Bengaluru-Chennai Expressway will significantly boost real estate development in certain areas, enhancing connectivity and driving property values.\n\nInfrastructure Developments and Connectivity in Bangalore\nThe city is undergoing significant infrastructure development that will enhance connectivity and support its ever-expanding population. Let us explore some of the key projects in 2024 and beyond:\